In [1]:
import os
import sys
import pandas as pd
import multiprocessing
from tqdm import tqdm

from utils.read_csv import *

In [2]:
### Global variables ###    
data_path = '../datasets/highd-dataset-v1.0/data/'
out_path = './outputs/tracks-slurm/'
os.makedirs(out_path, exist_ok=True)

In [3]:
def main(i):

    idx_str = '{0:02}'.format(i)

    created_arguments = {
        'input_path': data_path + idx_str + "_tracks.csv",
        'input_static_path': data_path + idx_str + "_tracksMeta.csv",
        'input_meta_path': data_path + idx_str + "_recordingMeta.csv",
        'background_image': data_path + idx_str + "_highway.jpg",
    }

    ### Read the video meta ###
    df_rec = pd.read_csv(created_arguments["input_meta_path"])
    df_rec.to_csv(out_path + idx_str + '_meta.csv', index=False)
    df_rec.to_pickle(out_path + idx_str + '_meta.pkl')
#    meta = pd.read_pickle(out_path + idx_str + '_meta.pkl')
#    display(meta)

    ### Read the static info ###
    try:
        static_info = read_static_info(created_arguments)
    except:
        print("The static info file is either missing or contains incorrect characters.")
        sys.exit(1)
    df_stc = pd.DataFrame(static_info).T
#    display(df_stc)

    ### Read the track csv and convert to useful format ###
    tracks = read_track_csv(created_arguments, idx_str, df_stc, df_rec)
    df_trc = pd.DataFrame(tracks)
#    display(df_trc)

    ### Merge data ###
    df_comb = df_stc.merge(df_trc, on='id')
    df_comb.to_csv(out_path + idx_str + '_data.csv', index=False)
    df_comb.to_pickle(out_path + idx_str + '_data.pkl')
#    data = pd.read_pickle(out_path + idx_str + '_data.pkl')
#    display(data)

In [ ]:
# iter = list(range(1, 61))
iter = list(range(1, 2))
for i in tqdm(iter):
    # print('Which recording is under test? '+str(i))
    main(i)

In [4]:
# Combine data from different recordings
df_data = pd.DataFrame()
df_meta = pd.DataFrame()
for i in tqdm(range(1, 61)):
    idx_str = '{0:02}'.format(i)
    df1 = pd.read_pickle(out_path + idx_str + '_data.pkl')
    df1.insert(loc=0, column='recordingId', value=i)
    df_data = pd.concat([df_data, df1])

    df2 = pd.read_pickle(out_path + idx_str + '_meta.pkl')
    df_meta = pd.concat([df_meta, df2])

df_data =  df_data.rename(columns={'id': 'vehicleId'})
# df_data.to_pickle(out_path + 'data.pkl')
# df_data.to_csv(out_path + 'data.csv', index=False)

df_meta = df_meta.rename(columns={'id': 'recordingId'})
df_meta.to_pickle(out_path + 'meta.pkl')
df_meta.to_csv(out_path + 'meta.csv', index=False)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [00:44<00:00,  1.36it/s]
